## Phase 1: Demand Modeling

### I. Environment Setup

In [2]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [3]:
!mkdir -p /content/drive/MyDrive/Spring2025/TH/256-project/nhanes/2017-2018
!cd /content/drive/MyDrive/Spring2025/TH/256-project/nhanes/2017-2018
!wget -P /content/drive/MyDrive/Spring2025/TH/256-project/nhanes/2017-2018 https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/DEMO_J.XPT
!wget -P /content/drive/MyDrive/Spring2025/TH/256-project/nhanes/2017-2018 https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/DIQ_J.XPT

--2025-05-12 05:33:46--  https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/DEMO_J.XPT
Resolving wwwn.cdc.gov (wwwn.cdc.gov)... 13.107.246.40, 2620:1ec:bdf::40
Connecting to wwwn.cdc.gov (wwwn.cdc.gov)|13.107.246.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3412720 (3.3M) [text/plain]
Saving to: ‘/content/drive/MyDrive/Spring2025/TH/256-project/nhanes/2017-2018/DEMO_J.XPT.3’

DEMO_J.XPT.3        100%[===================>]   3.25M  16.8MB/s    in 0.2s    

2025-05-12 05:33:46 (16.8 MB/s) - ‘/content/drive/MyDrive/Spring2025/TH/256-project/nhanes/2017-2018/DEMO_J.XPT.3’ saved [3412720/3412720]

--2025-05-12 05:33:46--  https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/DIQ_J.XPT
Resolving wwwn.cdc.gov (wwwn.cdc.gov)... 13.107.246.40, 2620:1ec:bdf::40
Connecting to wwwn.cdc.gov (wwwn.cdc.gov)|13.107.246.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3851840 (3.7M) [text/plain]
Saving to: ‘/content/drive/MyDriv

In [4]:
!pip install nhanes pandas numpy statsmodels scikit-learn plotly ucimlrepo
!pip install --upgrade pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 72.7 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


### II. Data Acquisition

##### 1. NHANES 2017-2018

Source: CDC's continuous NHANES site provides all public-use files for 2017-2018 CDC.

Python access via the nhanes package (interface patterned after the R NHANES package) GitHub.

##### 2. Diabetes 130-US Hospitals (1999-2008)

Source: UCI Machine Learning Repository, "Diabetes 130-US Hospitals for Years 1999-2008" UCI Machine Learning Repository.

Dataset details: 101,766 records, 47 features including patient demographics, lab tests, medications, length of stay, and 30-day readmission.

In [5]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
# NHANES 2017–2018
demo_path = '/content/drive/MyDrive/Spring2025/TH/256-project/nhanes/2017-2018/DEMO_J.XPT'
diq_path  = '/content/drive/MyDrive/Spring2025/TH/256-project/nhanes/2017-2018/DIQ_J.XPT'
# format='xport' is inferred but can be set explicitly
demo = pd.read_sas(demo_path, format='xport')  # Demographics
diq  = pd.read_sas(diq_path,  format='xport')  # Diabetes questionnaire
# Diabetes 130-US Hospitals (1999-2008)
diabetes_130_us_hospitals_for_years_1999_2008 = fetch_ucirepo(id=296)
uci_X = diabetes_130_us_hospitals_for_years_1999_2008.data.features
uci_y = diabetes_130_us_hospitals_for_years_1999_2008.data.targets


/usr/local/lib/python3.11/dist-packages/ucimlrepo/fetch.py:97: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


### III. Processing

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from ucimlrepo import fetch_ucirepo
import matplotlib.pyplot as plt
import seaborn as sns


Step 1: Merge & Clean NHANES

In [7]:
nhanes = pd.merge(demo, diq, on='SEQN', how='inner')
# adult
nhanes = nhanes[nhanes['RIDAGEYR'] >= 18]
nhanes = nhanes[nhanes['DIQ010'].isin([1, 2])]
nhanes['has_diabetes'] = nhanes['DIQ010'] == 1
nhanes['age_bin'] = pd.cut(
    nhanes['RIDAGEYR'],
    bins=[18, 30, 45, 60, 75, 90],
    labels=['18–30', '31–45', '46–60', '61–75', '76+'])


Step 2: Aggregate NHANES Diabetes Prevalence

In [8]:
region_proxy = 'SDMVSTRA'  # demographic proxy
grouped = (
    nhanes
    .groupby([region_proxy, 'age_bin'])['has_diabetes']
    .agg(prevalence='mean', n='count')
    .reset_index()
)


<ipython-input-8-870f169329bb>:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby([region_proxy, 'age_bin'])['has_diabetes']


Step 3: Load UCI Diabetes 130-US Hospital Dataset

In [9]:
uci = pd.concat([uci_X, uci_y], axis=1)
uci = uci[uci['race'] != '?']  # Clean unknowns
uci['readmitted_flag'] = uci['readmitted'].apply(lambda x: 1 if x == '<30' else 0) # # 2.2 Recode readmission into binary flag (<30 days=1, else=0)


Step 4: Aggregate UCI

In [10]:
hospital_burden = (
    uci
    .groupby('age')['readmitted_flag']
    .agg(readmission_rate='mean', n='count')
    .reset_index()
)
hospital_burden.rename(columns={'mean': 'readmission_rate', 'count': 'n'}, inplace=True)


Step 5: PCA z-score

In [11]:
def zscore_transform(*arrays):
    stacked = np.vstack([np.asarray(arr).reshape(-1) for arr in arrays]).T
    scaler = StandardScaler()
    scaled = scaler.fit_transform(stacked)
    return [scaled[:, i] for i in range(scaled.shape[1])]

def pca_composite(*zarrays, n_components=1):
    X = np.vstack([np.asarray(z).reshape(-1) for z in zarrays]).T
    pca = PCA(n_components=n_components)
    pcs = pca.fit_transform(X)
    loadings = pca.components_[0]
    weights = loadings / np.sum(loadings)
    return pcs[:, 0], weights


Step 6: Compute z‑scores for each region×age_bin’s diabetes_prevalence
(and—optionally—hospital readmission by mapping age_bin→age)

In [12]:
prev = grouped['prevalence'].values
# If you align hospital_burden to the same bins, extract its rate vector `rb`
# For illustration, we z‑score only NHANES prevalence here:
[z_prev] = zscore_transform(prev)

In [13]:
std_pop = {
  '18–30': 0.236,
  '31–45': 0.293,
  '46–60': 0.253,
  '61–75': 0.137,
  '76+':   0.082
} # https://www.cdc.gov/nchs/hus/sources-definitions/age-adjustment.html


In [14]:
def age_adjusted(df, weight_map):
    df = df.copy()
    df['age_bin_str'] = df['age_bin'].astype(str)
    df['std_weight'] = df['age_bin_str'].map(weight_map).astype(float)
    df['weighted_prev'] = df['prevalence'] * df['std_weight']
    return df.groupby(region_proxy)['weighted_prev'].sum().reset_index(
        name='age_adj_prevalence'
    )

age_std = age_adjusted(grouped, std_pop)
age_std.head()


,SDMVSTRA,age_adj_prevalence
0,134.0,0.099036
1,135.0,0.128388
2,136.0,0.136306
3,137.0,0.131166
4,138.0,0.136666


In [15]:
grouped.to_csv('/content/drive/MyDrive/Spring2025/TH/256-project/output/nhanes_grouped.csv', index=False)
age_std.to_csv('/content/drive/MyDrive/Spring2025/TH/256-project/output/nhanes_age_std.csv', index=False)


Build composite score via PCA (if combining multiple indicators)

In [16]:
composite, weights = pca_composite(z_prev)
grouped['composite_burden'] = composite

print("PCA weights:", weights)
grouped.head()

PCA weights: [1.]


,SDMVSTRA,age_bin,prevalence,n,composite_burden
0,134.0,18–30,0.000000,47,-1.307399
1,134.0,31–45,0.031746,63,-1.057528
2,134.0,46–60,0.141026,78,-0.197394
3,134.0,61–75,0.209302,86,0.340008
4,134.0,76+,0.309524,42,1.128845


### IV. Age-Standardized Prevalence Mapping

Step 1: Load nhanes_grouped.csv and nhanes_age_std.csv

In [17]:
import pandas as pd
import plotly.express as px

grouped = pd.read_csv('/content/drive/MyDrive/Spring2025/TH/256-project/output/nhanes_grouped.csv')
age_std = pd.read_csv('/content/drive/MyDrive/Spring2025/TH/256-project/output/nhanes_age_std.csv')


In [18]:
print("Unique region codes in grouped:", grouped['SDMVSTRA'].unique())
print("Unique region codes in age_std:", age_std['SDMVSTRA'].unique())


Unique region codes in grouped: [134. 135. 136. 137. 138. 139. 140. 141. 142. 143. 144. 145. 146. 147.
 148.]
Unique region codes in age_std: [134. 135. 136. 137. 138. 139. 140. 141. 142. 143. 144. 145. 146. 147.
 148.]


Step 2: Apply mapping

In [19]:
strata_to_region = {
    134: 'Northeast', 135: 'Northeast', 136: 'Northeast', 137: 'Northeast',
    138: 'Midwest', 139: 'Midwest', 140: 'Midwest', 141: 'Midwest',
    142: 'South', 143: 'South', 144: 'South', 145: 'South',
    146: 'West', 147: 'West', 148: 'West', 149: 'West',
    150: 'West'
}
# Map survey strata codes to state abbreviations

age_std['SDMVSTRA'] = age_std['SDMVSTRA'].astype(int)
age_std['region'] = age_std['SDMVSTRA'].map(strata_to_region)

In [20]:
print(age_std[['SDMVSTRA','age_adj_prevalence']].head(10))


   SDMVSTRA  age_adj_prevalence
0       134            0.099036
1       135            0.128388
2       136            0.136306
3       137            0.131166
4       138            0.136666
5       139            0.139837
6       140            0.100180
7       141            0.122098
8       142            0.121626
9       143            0.147801


In [21]:
region_prev = (
    age_std
    .groupby('region')['age_adj_prevalence']
    .mean()
    .reset_index(name='mean_age_std_prev')
)
print(region_prev)


      region  mean_age_std_prev
0    Midwest           0.124695
1  Northeast           0.123724
2      South           0.117349
3       West           0.143699


Step 5: Plotly choropleth of age-standardized prevalence and style adjustments

In [22]:
state_to_region = {
    # Northeast
    'CT':'Northeast','ME':'Northeast','MA':'Northeast','NH':'Northeast',
    'RI':'Northeast','VT':'Northeast','NJ':'Northeast','NY':'Northeast',
    'PA':'Northeast',
    # Midwest
    'IL':'Midwest','IN':'Midwest','MI':'Midwest','OH':'Midwest','WI':'Midwest',
    'IA':'Midwest','KS':'Midwest','MN':'Midwest','MO':'Midwest','NE':'Midwest',
    'ND':'Midwest','SD':'Midwest',
    # South
    'DE':'South','FL':'South','GA':'South','MD':'South','NC':'South','SC':'South',
    'VA':'South','DC':'South','WV':'South','AL':'South','KY':'South','MS':'South',
    'TN':'South','AR':'South','LA':'South','OK':'South','TX':'South',
    # West
    'AZ':'West','CO':'West','ID':'West','MT':'West','NV':'West','NM':'West',
    'UT':'West','WY':'West','AK':'West','CA':'West','HI':'West','OR':'West',
    'WA':'West'
}

# Step 1: Assign region to each state
state_region_df = pd.DataFrame({'state': list(state_to_region.keys())})
state_region_df['region'] = state_region_df['state'].map(state_to_region)

# Step 2: Merge with region prevalence
state_region_prev = state_region_df.merge(region_prev, on='region', how='left')

# Step 3: Plot!
fig = px.choropleth(
    state_region_prev,
    locations='state',
    locationmode='USA-states',
    color='mean_age_std_prev',
    scope='usa',
    hover_data=['region', 'mean_age_std_prev'],
    labels={'mean_age_std_prev': 'Age-Std Prevalence'},
    title='Age-Standardized Diabetes Prevalence by Region (NHANES 2017–2018)'
)

fig.update_layout(
    geo=dict(showlakes=True, lakecolor='lightblue'),
    margin=dict(l=0, r=0, t=50, b=0),
    coloraxis_colorbar=dict(title="Prevalence")
)

fig.show()

## Phase 2: Modified Matching Algorithm

In [24]:
# # combined appraoch
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import MinMaxScaler

# # — Regions & burden from Phase 1 —
# regions = region_prev['region'].tolist()
# burden = region_prev.set_index('region')['mean_age_std_prev']

# # 1. Simulate 2,000 physicians with attributes influencing rural affinity
# np.random.seed(0)
# n_physicians = 2000
# physicians = pd.DataFrame({
#     'physician_id': [f'doc_{i+1}' for i in range(n_physicians)],
#     'upbringing_rural': np.random.binomial(1, 0.2, n_physicians),   # 20% rural-raised :contentReference[oaicite:0]{index=0}
#     'rural_training':  np.random.binomial(1, 0.3, n_physicians),   # 30% rural rotations :contentReference[oaicite:1]{index=1}
#     'family_ties':     np.random.binomial(1, 0.25, n_physicians),  # 25% rural family ties :contentReference[oaicite:2]{index=2}
# })

# # 2. Build base preference scores per region
# rows = []
# for _, doc in physicians.iterrows():
#     for r in regions:
#         is_rural_region = int(r in ['South', 'West'])
#         noise = np.random.normal(scale=0.1)
#         base_score = (
#             0.4 * doc.upbringing_rural * is_rural_region +
#             0.3 * doc.rural_training   * is_rural_region +
#             0.2 * doc.family_ties +
#             noise
#         )
#         rows.append({'physician_id': doc.physician_id, 'region': r, 'base_pref': base_score})
# phys_pref = pd.DataFrame(rows)

# # 3. Normalize base preferences
# scaler = MinMaxScaler()
# phys_pref['norm_pref'] = scaler.fit_transform(phys_pref[['base_pref']])
# phys_pref.to_csv('/content/drive/MyDrive/Spring2025/TH/256-project/output/physician_pref.csv', index=False)

# # 4. Incorporate HPSA-style priority: boost burden for underserved regions
# #    Based on HRSA methodology: population-to-provider ratio, poverty, travel time :contentReference[oaicite:3]{index=3}
# hpsa_regions = ['South', 'West']
# region_prev['hpsa_priority'] = region_prev['region'].isin(hpsa_regions).astype(int)
# region_prev['adj_burden'] = region_prev['mean_age_std_prev'] * (1 + 0.25 * region_prev['hpsa_priority'])

# # 5. Normalize adjusted burden
# burden_norm = scaler.fit_transform(region_prev[['adj_burden']]).flatten()
# burden_df = pd.DataFrame({'region': regions, 'norm_burden': burden_norm})

# # 6. Merge and compute final match score
# match_df = phys_pref.merge(burden_df, on='region')
# match_df['match_score'] = 0.4 * match_df['norm_pref'] + 0.6 * match_df['norm_burden']

# # 7. Assign each physician to the top-scoring region
# top_matches = (
#     match_df
#     .sort_values('match_score', ascending=False)
#     .groupby('physician_id')
#     .first()
#     .reset_index()[['physician_id', 'region', 'match_score']]
# )

# print(top_matches.head())


  physician_id region  match_score
0        doc_1   West     0.813124
1       doc_10   West     0.759697
2      doc_100   West     0.724644
3     doc_1000   West     0.768973
4     doc_1001   West     0.676928


In [ ]:
# realistic approach
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import MinMaxScaler

# # Regions & burden from Phase 1
# regions = region_prev['region'].tolist()
# burden = region_prev.set_index('region')['mean_age_std_prev']

# # 1. Simulate physician attributes
# np.random.seed(0)
# n = 2000
# physicians = pd.DataFrame({
#     'physician_id': [f'doc_{i+1}' for i in range(n)],
#     'upbringing_rural': np.random.binomial(1, 0.2, n),   # 20% rural-raised
#     'rural_training':  np.random.binomial(1, 0.3, n),   # 30% had rural rotations
#     'family_ties':     np.random.binomial(1, 0.25, n),  # 25% have rural spouse/family
# })

# # 2. Build physician–region preference scores
# rows = []
# for _, doc in physicians.iterrows():
#     for r in regions:
#         is_rural_region = 1 if r in ['South','West'] else 0
#         # weights
#         w_u, w_t, w_s = 0.4, 0.3, 0.2
#         noise = np.random.normal(scale=0.1)
#         base_score = (
#             w_u * doc.upbringing_rural * is_rural_region +
#             w_t * doc.rural_training   * is_rural_region +
#             w_s * doc.family_ties +
#             noise
#         )
#         rows.append({
#             'physician_id': doc.physician_id,
#             'region': r,
#             'base_pref': base_score
#         })
# phys_pref = pd.DataFrame(rows)

# # 3. Normalize preferences & burden
# scaler = MinMaxScaler()
# phys_pref['norm_pref'] = scaler.fit_transform(phys_pref[['base_pref']])
# burden_norm = scaler.fit_transform(burden.values.reshape(-1,1)).flatten()
# burden_df = pd.DataFrame({'region': regions, 'norm_burden': burden_norm})

# # 4. Merge & compute final match score
# match_df = phys_pref.merge(burden_df, on='region')
# match_df['match_score'] = 0.4*match_df['norm_pref'] + 0.6*match_df['norm_burden']

# # 5. Assign each physician to top region
# top_matches_realistic = (
#     match_df.sort_values('match_score', ascending=False)
#             .groupby('physician_id')
#             .first()
#             .reset_index()[['physician_id','region','match_score']]
# )
# print(top_matches_realistic.head())


  physician_id region  match_score
0        doc_1   West     0.813124
1       doc_10   West     0.759697
2      doc_100   West     0.724644
3     doc_1000   West     0.768973
4     doc_1001   West     0.676928


#### Fine-tune parameters

In [23]:
n_physicians = 6000
HPSA_WEIGHT = 4.752
arpq_short_term_diabetes_PQI01_rate_2018 = 81.80
physician_ids = [f'doc_{i+1}' for i in range(n_physicians)]
regions = region_prev['region'].unique()

### I. Simulate Physician Preference

In [ ]:
import numpy as np
import pandas as pd

# Simulate physician preferences: each physician ranks regions (0 to 1 scale)
np.random.seed(42)
physician_prefs = pd.DataFrame({
    'physician_id': np.repeat(physician_ids, len(regions)),
    'region': np.tile(regions, n_physicians),
    'base_pref': np.random.rand(n_physicians * len(regions))
})
physician_prefs.to_csv('/content/drive/MyDrive/Spring2025/TH/256-project/output/physician_pref.csv', index=False)


In [ ]:
# Merge physician preferences with regional disease burden
match_df = physician_prefs.merge(region_prev, on='region', how='left')

# Normalize both components for scoring
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
match_df[['norm_pref', 'norm_burden']] = scaler.fit_transform(
    match_df[['base_pref', 'mean_age_std_prev']]
)

# Compute composite match score
match_df['match_score'] = 0.4 * match_df['norm_pref'] + 0.6 * match_df['norm_burden']

### II. Create Algorithm

In [ ]:
# Get top region for each physician
top_matches = (
    match_df.sort_values(by='match_score', ascending=False)
            .groupby('physician_id')
            .first()
            .reset_index()
)

# Output the physician-region matches
print(top_matches[['physician_id', 'region', 'match_score']].head())


  physician_id region  match_score
0        doc_1   West     0.839480
1       doc_10   West     0.776072
2      doc_100   West     0.901839
3     doc_1000   West     0.664548
4     doc_1001   West     0.661561


### III. Add HPSA-Like Weights

In [ ]:
# Simulate HPSA-style binary underserved tag (could be replaced with real data)
hpsa_regions = ['South', 'West']  # Example underserved regions
region_prev['hpsa_priority'] = region_prev['region'].isin(hpsa_regions).astype(int)

# Boost disease burden for underserved areas
region_prev['adjusted_burden'] = (
    region_prev['mean_age_std_prev'] * (1 + HPSA_WEIGHT * region_prev['hpsa_priority'])
)
region_prev.to_csv('/content/drive/MyDrive/Spring2025/TH/256-project/output/region_prev.csv', index=False)

# Recompute normalized and composite score with adjustment
match_df = match_df.drop(columns=['norm_burden', 'match_score'])
match_df = match_df.merge(region_prev[['region', 'adjusted_burden']], on='region', how='left')

match_df[['norm_pref', 'norm_burden']] = scaler.fit_transform(
    match_df[['preference', 'adjusted_burden']]
)
match_df['match_score'] = 0.4 * match_df['norm_pref'] + 0.6 * match_df['norm_burden']


### IV. Rematch With An Adjusted Score

In [ ]:
# Assign physicians again using adjusted scores
top_matches_adjusted = (
    match_df.sort_values(by='match_score', ascending=False)
            .groupby('physician_id')
            .first()
            .reset_index()
)
top_matches_adjusted.to_csv('/content/drive/MyDrive/Spring2025/TH/256-project/output/top_matches_adjusted.csv', index=False)

print(top_matches_adjusted[['physician_id', 'region', 'match_score']].head())


  physician_id region  match_score
0        doc_1   West     0.839480
1       doc_10   West     0.776072
2      doc_100   West     0.901839
3     doc_1000  South     0.682597
4     doc_1001  South     0.774693


In [ ]:
# Compare top region matches before and after adjustment
comparison = top_matches[['physician_id', 'region']].merge(
    top_matches_adjusted[['physician_id', 'region']],
    on='physician_id',
    suffixes=('_before', '_after')
)

# See which changed
comparison['changed'] = comparison['region_before'] != comparison['region_after']
print(comparison['changed'].value_counts())


changed
False    4612
True     1388
Name: count, dtype: int64


## Phase 3: Policy Simulation

In [26]:
import pandas as pd
import numpy as np

# Load top_matches from Phase 2
top_matches = pd.read_csv('/content/drive/MyDrive/Spring2025/TH/256-project/output/top_matches_adjusted.csv')

# Count physicians per region
physician_counts = top_matches['region'].value_counts().reset_index()
physician_counts.columns = ['region', 'physician_count']

# Merge with regional burden data
region_prev = pd.read_csv('/content/drive/MyDrive/Spring2025/TH/256-project/output/region_prev.csv')
region_data = pd.merge(region_prev, physician_counts, on='region', how='left').fillna(0)

# Assume baseline hospitalizations per region (e.g., from historical data)
# For demonstration, we'll use a placeholder value
region_data['baseline_hospitalizations'] = 1000  # Replace with actual data

# Calculate expected hospitalizations after reallocation
# Assume a 1% reduction in hospitalizations per additional physician
region_data['expected_hospitalizations'] = region_data['baseline_hospitalizations'] * (1 - 0.01 * region_data['physician_count'])

# Calculate avoidable hospitalizations
region_data['avoidable_hospitalizations'] = region_data['baseline_hospitalizations'] - region_data['expected_hospitalizations']

# Save results
region_data.to_csv('/content/drive/MyDrive/Spring2025/TH/256-project/output/avoidable_hospitalizations.csv', index=False)


In [27]:
# Random Matching
random_matches = top_matches.copy()
random_matches['region'] = np.random.choice(region_prev['region'], size=len(top_matches))

# Preference-Only Matching
# Assuming 'phys_pref' DataFrame contains physician preferences
phys_pref = pd.read_csv('/content/drive/MyDrive/Spring2025/TH/256-project/output/physician_pref.csv')
preference_only_matches = phys_pref.loc[phys_pref.groupby('physician_id')['norm_pref'].idxmax()][['physician_id', 'region']]

# Function to calculate avoidable hospitalizations for a given matching
def calculate_avoidable_hospitalizations(matches, method_name):
    physician_counts = matches['region'].value_counts().reset_index()
    physician_counts.columns = ['region', 'physician_count']
    data = pd.merge(region_prev, physician_counts, on='region', how='left').fillna(0)
    data['baseline_hospitalizations'] = 1000  # Replace with actual data
    data['expected_hospitalizations'] = data['baseline_hospitalizations'] * (1 - 0.01 * data['physician_count'])
    data['avoidable_hospitalizations'] = data['baseline_hospitalizations'] - data['expected_hospitalizations']
    data['method'] = method_name
    return data[['region', 'avoidable_hospitalizations', 'method']]

# Calculate for each method
proposed_data = calculate_avoidable_hospitalizations(top_matches, 'Proposed')
random_data = calculate_avoidable_hospitalizations(random_matches, 'Random')
preference_data = calculate_avoidable_hospitalizations(preference_only_matches, 'Preference-Only')

# Combine results
comparison_data = pd.concat([proposed_data, random_data, preference_data])
comparison_data.to_csv('/content/drive/MyDrive/Spring2025/TH/256-project/output/comparison_data.csv', index=False)


In [28]:
import plotly.express as px

# Load comparison data
comparison_data = pd.read_csv('/content/drive/MyDrive/Spring2025/TH/256-project/output/comparison_data.csv')

# Bar chart comparison
fig = px.bar(comparison_data, x='region', y='avoidable_hospitalizations', color='method',
             title='Avoidable Hospitalizations by Region and Matching Method',
             labels={'avoidable_hospitalizations': 'Avoidable Hospitalizations', 'region': 'Region'})
fig.show()


### I. Data Preparation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load region prevalence & matches

# Load age-standardized prevalence by region
age_std = pd.read_csv('/content/drive/MyDrive/Spring2025/TH/256-project/output/region_prev.csv')

# Load physician-region matches (demand-informed)
top_matches = pd.read_csv('/content/drive/MyDrive/Spring2025/TH/256-project/output/top_matches_adjusted.csv')


### II. Simulation

In [ ]:
arpq_short_term_diabetes_PQI01_rate_2018 = 81.80

In [ ]:
# 1. Map region populations
region_pop = {
    'Northeast': 57609148,
    'Midwest':   68985454,
    'South':    126266107,
    'West':      78588572
}

# 2. Compute baseline admissions per region
pqi_rate_per1000 = arpq_short_term_diabetes_PQI01_rate_2018 / 100  # convert per 100k → per 1k
region_prev['population'] = region_prev['region'].map(region_pop)
region_prev['baseline_adm'] = (
    region_prev['population'] * pqi_rate_per1000
)

# 3. Count assignments
n_physicians = top_matches['physician_id'].nunique()
regions = region_prev['region'].unique()
physician_prefs = pd.read_csv('/content/drive/MyDrive/Spring2025/TH/256-project/output/physician_pref.csv')

# Demand-informed (already top_matches)
assigned_demand = (
    top_matches.groupby('region')['physician_id']
               .nunique().reset_index(name='n_demand')
)

# Random assignment
random_assign = pd.DataFrame({
    'physician_id': physician_prefs['physician_id'].unique(),
    'region': np.random.choice(regions, size=n_physicians, replace=True)
})
assigned_random = (
    random_assign.groupby('region')['physician_id']
                 .nunique().reset_index(name='n_random')
)

# Preference-only assignment
pref_only = (
    physician_prefs.sort_values('base_pref', ascending=False)
                   .groupby('physician_id').first().reset_index()
)
assigned_pref = (
    pref_only.groupby('region')['physician_id']
             .nunique().reset_index(name='n_preference')
)

# 4. Merge & compute avoided hospitalizations
sim = (region_prev
       .merge(assigned_demand, on='region', how='left')
       .merge(assigned_random, on='region', how='left')
       .merge(assigned_pref, on='region', how='left')
       .fillna(0))

impact = 0.33
for col in ['n_demand','n_random','n_preference']:
    sim[f'avoided_{col}'] = (
        sim['baseline_adm'] * impact * sim[col]
    )


In [ ]:
import plotly.express as px

# Prepare bar chart data
df_plot = pd.DataFrame({
    'Scenario': ['Demand-Informed','Random','Preference-Only'],
    'Avoidable_Admissions': [
        sim['avoided_n_demand'].sum(),
        sim['avoided_n_random'].sum(),
        sim['avoided_n_preference'].sum()
    ]
})

fig = px.bar(
    df_plot,
    x='Scenario',
    y='Avoidable_Admissions',
    title='Annual Avoidable Hospitalizations by Allocation Scenario',
    labels={'Avoidable_Admissions':'Avoidable Admissions'}
)
fig.update_layout(
    margin=dict(l=20, r=20, t=50, b=20),
    yaxis_tickformat=',.0f'
)
fig.show()
